In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ds-3-datathon-2025-car-evaluation/car_eval_test.csv
/kaggle/input/ds-3-datathon-2025-car-evaluation/car_eval_train.csv
/kaggle/input/ds-3-datathon-2025-car-evaluation/car_eval_sample_submission.csv


In [2]:
test_df = pd.read_csv("/kaggle/input/ds-3-datathon-2025-car-evaluation/car_eval_test.csv")
train_df = pd.read_csv("/kaggle/input/ds-3-datathon-2025-car-evaluation/car_eval_train.csv")
example_df = pd.read_csv("/kaggle/input/ds-3-datathon-2025-car-evaluation/car_eval_sample_submission.csv")

In [53]:
train_df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,med,med,3,4,med,med,acc
1,med,high,3,2,small,med,unacc
2,low,high,5more,more,small,med,acc
3,med,low,2,4,med,high,good
4,vhigh,low,3,2,small,high,unacc


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1382 entries, 0 to 1381
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1382 non-null   object
 1   maint     1382 non-null   object
 2   doors     1382 non-null   object
 3   persons   1382 non-null   object
 4   lug_boot  1382 non-null   object
 5   safety    1382 non-null   object
 6   class     1382 non-null   object
dtypes: object(7)
memory usage: 75.7+ KB


In [47]:
train_df['doors'].unique()

array(['3', '5more', '2', '4'], dtype=object)

In [50]:
train_df['buying'].unique()

array(['med', 'low', 'vhigh', 'high'], dtype=object)

In [49]:
train_df['lug_boot'].unique()

array(['med', 'small', 'big'], dtype=object)

In [51]:
train_df['maint'].unique()

array(['med', 'high', 'low', 'vhigh'], dtype=object)

In [52]:
train_df['class'].unique()

array(['acc', 'unacc', 'good', 'vgood'], dtype=object)

In [58]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Load the data
train_df = pd.read_csv("/kaggle/input/ds-3-datathon-2025-car-evaluation/car_eval_train.csv")
test_df = pd.read_csv("/kaggle/input/ds-3-datathon-2025-car-evaluation/car_eval_test.csv")
example_df = pd.read_csv("/kaggle/input/ds-3-datathon-2025-car-evaluation/car_eval_sample_submission.csv")

# Step 1: Encoding categorical features using LabelEncoder
label_encoders = {}

# Create a copy of train_df and encode it
train_df_copy = train_df.copy()
for column in train_df_copy.columns:
    if train_df_copy[column].dtype == 'object':
        le = LabelEncoder()
        train_df_copy[column] = le.fit_transform(train_df_copy[column])
        label_encoders[column] = le

# Create a copy of test_df and apply the same encoding
test_df_copy = test_df.copy()
for column in test_df_copy.columns:
    if column in label_encoders:
        le = label_encoders[column]
        test_df_copy[column] = le.transform(test_df_copy[column])

# Step 2: Separate features and target variable in train_df_copy
X_train = train_df_copy.drop(columns=['class'])  # Features
y_train = train_df_copy['class']  # Target variable
print(X_train.head())
print(y_train.head())
# Step 3: Train a Random Forest Classifier with cross-validation
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform cross-validation to get a better estimate of accuracy
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy Scores: {cv_scores}")
print(f"Mean Cross-Validation Accuracy: {cv_scores.mean():.4f}")

# Step 4: Train the model on the entire training data and predict on the test set
rf_model.fit(X_train, y_train)

# Step 5: Predict on the test set (test_df_copy)
X_test = test_df_copy.drop(columns=["id"])  # Drop id column (if it exists)
y_test_pred = rf_model.predict(X_test)

# Step 6: Create a new DataFrame for predictions with the 'id' column intact
submission = test_df[['id']].copy()  # Keep the id column from test_df
# Convert numeric predictions back to the original class labels
class_labels = ['acc', 'good', 'unacc', 'vgood']  # Define the original class labels
submission['output'] = [class_labels[label] for label in y_test_pred]  # Map numeric labels to original class labels

# Step 7: Save the predictions to a CSV file
submission.to_csv('/kaggle/working/car_sample_submission.csv', index=False)

print("Submission file saved!")


   buying  maint  doors  persons  lug_boot  safety
0       2      2      1        1         1       2
1       2      0      1        0         2       2
2       1      0      3        2         2       2
3       2      1      0        1         1       0
4       3      1      1        0         2       0
0    0
1    2
2    0
3    1
4    2
Name: class, dtype: int64
Cross-Validation Accuracy Scores: [0.96750903 0.98194946 0.94202899 0.96376812 0.96014493]
Mean Cross-Validation Accuracy: 0.9631
Submission file saved!
